# 다양한 스트리밍 방식

간단한 그래프 정의

In [5]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional

class GoogleNews:
    """
    구글 뉴스를 검색하고 결과를 반환하는 클래스입니다.
    """

    def __init__(self):
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:

        if not news_list:
            print("해당 키워드의 뉴스가 없습니다.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=ko&gl=KR&ceid=KR:ko"
        else:
            url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

In [7]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_keyword])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()


1. 단순 동기 스트리밍

In [8]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="values")
for chunk in graph.stream(inputs, stream_mode="values"):

    # chunk 는 dictionary 형태(key: State 의 key, value: State 의 value)
    for state_key, state_value in chunk.items():
        if state_key == "messages":
            state_value[-1].pretty_print()


================================ Human Message =================================

AI 관련된 최신 뉴스를 검색해줘
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_rmMkbD8nOZZfpjUzRm4MnU0i)
 Call ID: call_rmMkbD8nOZZfpjUzRm4MnU0i
  Args:
    query: AI
================================= Tool Message =================================
Name: search_keyword

- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레
- [쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다" - 조선일보
- 네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스" - SBS 뉴스
- 파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원 - 디지털투데이
================================== Ai Message ==================================

최신 AI 관련 뉴스는 다음과 같습니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스"

2. 단순 비동기 스트리밍

In [9]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 비동기 스트림 처리(stream_mode="values")
async for chunk in graph.astream(inputs, stream_mode="values"):
    # chunk 는 dictionary 형태(key: State 의 key, value: State 의 value)
    for state_key, state_value in chunk.items():
        if state_key == "messages":
            state_value[-1].pretty_print()


================================ Human Message =================================

AI 관련된 최신 뉴스를 검색해줘
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_AyqpG130hYfN8fM3kjyuOQ3H)
 Call ID: call_AyqpG130hYfN8fM3kjyuOQ3H
  Args:
    query: AI
================================= Tool Message =================================
Name: search_keyword

- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레
- [쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다" - 조선일보
- 네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스" - SBS 뉴스
- 파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원 - 디지털투데이
================================== Ai Message ==================================

다음은 AI 관련 최신 뉴스입니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스"** 

3. 스트리밍 최종 결과만 확인하고 싶을 때

In [10]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

final_result = None

# 비동기 스트림 처리(stream_mode="values")
async for chunk in graph.astream(inputs, stream_mode="values"):
    final_result = chunk

# 최종 결과 출력
print(final_result["messages"][-1].content)


최신 AI 관련 뉴스는 다음과 같습니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스"** - SBS 뉴스
5. **파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원** - 디지털투데이

이 뉴스들은 AI 기술의 발전, 상용화, 그리고 관련 산업의 변화에 대해 다루고 있습니다.


stream mode가 updates 면은 새로 생긴 상태만 출력이 된다.

In [11]:
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="updates")
for chunk in graph.stream(inputs, stream_mode="updates"):
    # chunk 는 dictionary 형태(key: 노드, value: 노드의 상태 값)
    for node, value in chunk.items():
        if node:
            print(f"\n[Node: {node}]\n")
        if "messages" in value:
            value["messages"][-1].pretty_print()


[Node: chatbot]

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_thF1FKThIhw5X1AUxVRZeP7z)
 Call ID: call_thF1FKThIhw5X1AUxVRZeP7z
  Args:
    query: AI

[Node: tools]

================================= Tool Message =================================
Name: search_keyword

- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레
- [쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다" - 조선일보
- 네이버, 통합 콘퍼런스 DAN 24 개최…'AI 원천기술 밀착' 선언 - 전자신문
- 파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원 - 디지털투데이

[Node: chatbot]

================================== Ai Message ==================================

다음은 AI 관련 최신 뉴스입니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, 통합 콘퍼런스 DAN 24 개최…'AI 원천기술 밀착' 선언** - 전자신문
5. **파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품

messages는 각 단계 메시지만 출력함.

In [12]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="messages")
# chunk_msg: 실시간 출력 메시지, metadata: 노드 정보
for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages"):

    # chatbot 노드에서 출력된 메시지만 출력
    if metadata["langgraph_node"] == "chatbot":
        if chunk_msg.content:
            print(chunk_msg.content, end="", flush=True)

    else:
        print(chunk_msg.content)
        print(f"\n\nmetadata: \n{metadata}\n\n")


- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레
- [쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다" - 조선일보
- 네이버, 통합 콘퍼런스 DAN 24 개최…'AI 원천기술 밀착' 선언 - 전자신문
- 파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원 - 디지털투데이


metadata: 
{'langgraph_step': 2, 'langgraph_node': 'tools', 'langgraph_triggers': ['branch:chatbot:tools_condition:tools'], 'langgraph_path': ('__pregel_pull', 'tools'), 'langgraph_checkpoint_ns': 'tools:7712b7ff-5b40-d847-7bf2-56a960db427c'}


최근 AI 관련 뉴스는 다음과 같습니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, 통합 콘퍼런스 DAN 24 개최…'AI 원천기술 밀착' 선언** - 전자신문
5. **파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원** - 디지털투데이

더 궁금한 내용이 있다면 말씀해 주세요!

### 태그 사용 방법

In [21]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합 (tags 추가)
llm_with_tools = llm.bind_tools(tools).with_config(tags=["WANT_TO_STREAM"])

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_keyword])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()


In [ ]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 비동기 이벤트 스트림 처리(astream_events)
async for event in graph.astream_events(inputs, version="v2"):
    # 이벤트 종류와 태그 정보 추출
    kind = event["event"]
    tags = event.get("tags", [])
    print(event)
    # 채팅 모델 스트림 이벤트 및 최종 노드 태그 필터링
    if kind == "on_chat_model_stream" and "WANT_TO_STREAM" in tags:
        # 이벤트 데이터 추출
        data = event["data"]

        # 출력 메시지
        if data["chunk"].content:
            print(data["chunk"].content, end="", flush=True)


In [24]:
from langchain_core.messages import AIMessageChunk, HumanMessage

# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 첫 번째 메시지 처리 여부 플래그 설정
first = True

# 비동기 스트림 처리를 통한 메시지 및 메타데이터 순차 처리
for msg, metadata in graph.stream(inputs, stream_mode="messages"):
    # 사용자 메시지가 아닌 경우의 컨텐츠 출력 처리
    if msg.content and not isinstance(msg, HumanMessage):
        print(msg.content, end="", flush=True)

    # AI 메시지 청크 처리 및 누적
    if isinstance(msg, AIMessageChunk):
        if first:
            gathered = msg
            first = False
        else:
            gathered = gathered + msg

        # 도구 호출 청크 존재 시 누적된 도구 호출 정보 출력
        if msg.tool_call_chunks:
            print(gathered.tool_calls[0]["args"])


{}
{}
{}
{'query': ''}
{'query': 'AI'}
{'query': 'AI'}
- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’ - 한겨레
- [쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다" - 조선일보
- 네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스" - SBS 뉴스
- 파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원 - 디지털투데이다음은 AI 관련 최신 뉴스입니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **AI 교과서 ‘자율’→‘엄격 검정’ 돌변…출판사들 ‘졸속 후폭풍’** - 한겨레
3. **[쫌아는기자들] SBVA 이준표의 AI론, "대전의 스탠다드에너지는 게임체인저가 될 수도 있다"** - 조선일보
4. **네이버, AI 상용화 방안 공개…"내년 상반기 AI 검색 서비스"** - SBS 뉴스
5. **파인더스에이아이, ‘AI 자동 계산대’ 공개...1초 안에 상품 감지·결제 지원** - 디지털투데이

더 궁금한 점이 있으면 말씀해 주세요!

여러 그래프 합쳤을 때

In [26]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합 (tags 추가)
llm_with_tools = llm.bind_tools(tools).with_config(tags=["WANT_TO_STREAM"])

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# SNS 포스트 생성 함수 정의
def create_sns_post(state: State):
    # SNS 포스트 생성을 위한 프롬프트
    sns_prompt = """
    이전 대화 내용을 바탕으로 SNS 게시글 형식으로 변환해주세요.
    다음 형식을 따라주세요:
    - 해시태그 포함
    - 이모지 사용
    - 간결하고 흥미로운 문체 사용
    - 200자 이내로 작성
    """
    messages = state["messages"] + [("human", sns_prompt)]
    sns_llm = ChatOpenAI(model="gpt-4o-mini").with_config(tags=["WANT_TO_STREAM2"])
    return {"messages": [sns_llm.invoke(messages)]}

# 서브그래프 생성
def create_subgraph():
    # 서브그래프용 상태 그래프 생성 
    subgraph = StateGraph(State)

    # 챗봇 노드 추가
    subgraph.add_node("chatbot", chatbot)

    # 도구 노드 생성 및 추가
    tool_node = ToolNode(tools=[search_keyword])
    subgraph.add_node("tools", tool_node)

    # 조건부 엣지 추가
    subgraph.add_conditional_edges(
        "chatbot",
        tools_condition,
    )

    # tools > chatbot
    subgraph.add_edge("tools", "chatbot")

    # START > chatbot
    subgraph.add_edge(START, "chatbot")

    # chatbot > END 
    subgraph.add_edge("chatbot", END)

    return subgraph.compile()

# 메인 그래프 생성
graph_builder = StateGraph(State)

# 서브그래프 추가
subgraph = create_subgraph()
graph_builder.add_node("news_subgraph", subgraph)

# SNS 포스트 생성 노드 추가
graph_builder.add_node("sns_post", create_sns_post)

# START > news_subgraph
graph_builder.add_edge(START, "news_subgraph")

# news_subgraph > sns_post
graph_builder.add_edge("news_subgraph", "sns_post")

# sns_post > END
graph_builder.add_edge("sns_post", END)

# 그래프 컴파일
graph = graph_builder.compile()


서브 그래프 출력은 포함되지 않음.

In [27]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 노드 업데이트 정보 순차적 처리 및 출력
for chunk in graph.stream(inputs, stream_mode="updates"):
    # node_name: 현재 처리 중인 노드명, node_chunk: 해당 노드의 청크 데이터
    for node_name, node_chunk in chunk.items():
        # 현재 처리 중인 노드 구분선 출력
        print(f"\n========= Update from node {node_name} =========\n")
        # 해당 노드의 업데이트된 데이터 출력
        if "messages" in node_chunk:
            node_chunk["messages"][-1].pretty_print()
        else:
            print(node_chunk)




========= Update from node news_subgraph =========

================================== Ai Message ==================================

최근 AI 관련 뉴스는 다음과 같습니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **"인간 넘어서는 AI, 경제적 가치 1경달러…강력한 만큼 위험"** - 한국경제
3. **네이버, 내년 상반기 AI 검색 서비스‥"AI 원천기술 밀착 선언"** - MBC 뉴스
4. **중국 빅테크, AI 인력 확보 전쟁…"IT 평균의 2배 급여로 구애"** - 연합뉴스
5. **AI스마트광융복합협동조합, 광주국제고 AIoT실험 과학특강 성료** - 전기신문

더 궁금한 내용이 있으시면 말씀해 주세요!

========= Update from node sns_post =========

================================== Ai Message ==================================

🚀 최신 AI 뉴스! 🤖

오픈AI, '오라이온' 개선 위해 전략 수정 중! 💡 인간을 넘어서는 AI의 경제적 가치는 무려 1경 달러! 💰 네이버는 내년 AI 검색 서비스 출시 예정! 🔍 중국 빅테크는 AI 인력 확보 전쟁 중! 🥇

#AI #인공지능 #오픈AI #네이버 #기술혁신 #경제 #트렌드


만약 서브 그래프 출력도 필요하다면 subgraphs 파라미터를 True로 넣어야함.

In [28]:
# 사용자의 메시지를 딕셔너리 형태로 입력 데이터 구성
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 네임스페이스 문자열을 보기 좋은 형식으로 변환하는 포맷팅 함수
def format_namespace(namespace):
    return (
        namespace[-1].split(":")[0]
        if len(namespace) > 0
        else "parent graph"
    )

# subgraphs=True 를 통해 서브그래프의 출력도 포함(namespace, chunk) 형태로 출력됩니다.
for namespace, chunk in graph.stream(inputs, stream_mode="updates", subgraphs=True):
    # node_name: 현재 처리 중인 노드명, node_chunk: 해당 노드의 청크 데이터
    for node_name, node_chunk in chunk.items():
        print(f"\n========= Update from node [{node_name}] in [{format_namespace(namespace)}] =========\n")

        # 노드의 청크 데이터 출력
        if "messages" in node_chunk:
            node_chunk["messages"][-1].pretty_print()
        else:
            print(node_chunk)



========= Update from node [chatbot] in [news_subgraph] =========

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_WytvAshylxBbOvX9phT0pUDl)
 Call ID: call_WytvAshylxBbOvX9phT0pUDl
  Args:
    query: AI

========= Update from node [tools] in [news_subgraph] =========

================================= Tool Message =================================
Name: search_keyword

- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- "인간 넘어서는 AI, 경제적 가치 1경달러…강력한 만큼 위험" - 한국경제
- 네이버, 내년 상반기 AI 검색 서비스‥"AI 원천기술 밀착 선언" - MBC 뉴스
- 중국 빅테크, AI 인력 확보 전쟁…"IT 평균의 2배 급여로 구애" - 연합뉴스
- AI스마트광융복합협동조합, 광주국제고 AIoT실험 과학특강 성료 - 전기신문

========= Update from node [chatbot] in [news_subgraph] =========

================================== Ai Message ==================================

다음은 AI 관련 최신 뉴스입니다:

1. **"GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정** - AI타임스
2. **"인간 넘어서는 AI, 경제적 가치 1경달러…강력한 만큼 위험"** - 한국경제
3. **네이버, 내년 상반기 AI 검색 서비스‥"AI 원천기술 

event 타입별로 정리할 수도 있음.

존재하는 타입 예시는 : https://wikidocs.net/265576 

In [29]:
# 네임스페이스 정보를 파싱하는 함수
def parse_namespace_info(info: tuple) -> tuple[str, str]:
    if len(info) > 1:
        namespace, node_name = info
        return node_name.split(":")[0], namespace.split(":")[0]
    return info[0].split(":")[0], "parent graph"

kind = None

async for event in graph.astream_events(inputs, version="v2", subgraphs=True):
    kind = event["event"]

    # 이벤트 종류와 태그 정보 추출
    if kind == "on_chat_model_start":
        print(f"\n========= on_chat_model_start =========\n")

    # 채팅 모델 스트림 이벤트 및 최종 노드 태그 필터링
    elif kind == "on_chat_model_stream":
        # 이벤트 데이터 추출
        data = event["data"]

        # 토큰 단위의 스트리밍 출력
        if data["chunk"].content:
            print(data["chunk"].content, end="", flush=True)

    elif kind == "on_tool_start":
        print(f"\n========= tool_start =========\n")
        data = event["data"]
        if "input" in data:
            tool_msg = data["input"]
            print(tool_msg)            

    elif kind == "on_tool_end":
        print(f"\n========= tool_end =========\n")
        data = event["data"]
        if "output" in data:
            tool_msg = data["output"]
            print(tool_msg.content)




========= tool_start =========

{'query': 'AI'}

========= tool_end =========

- "GPT 성능 향상 속도 둔화"...오픈AI, '오라이온' 개선 위해 전략 수정 - AI타임스
- "인간 넘어서는 AI, 경제적 가치 1경달러…강력한 만큼 위험" - 한국경제
- 네이버, 내년 상반기 AI 검색 서비스‥"AI 원천기술 밀착 선언" - MBC 뉴스
- 중국 빅테크, AI 인력 확보 전쟁…"IT 평균의 2배 급여로 구애" - 연합뉴스
- AI스마트광융복합협동조합, 광주국제고 AIoT실험 과학특강 성료 - 전기신문
